# 다음 사항들을 바꿔가면서 학습 해보기

### 1) KoNLPy 토크나이저 클래스
- Mecab
    - 메카브, 일본어용 형태소 분석기를 한국어를 사용할 수 있도록 수정
    - https://bitbucket.org/eunjeon/mecab-ko
- Hannanum
    - 한나눔, KAIST Semantic Web Research Center 개발
    - http://semanticweb.kaist.ac.kr/hannanum/
- Kkma
    - 꼬꼬마, 서울대학교 IDS(Intelligent Data System)연구실 개발
    - http://kkma.snu.ac.kr/
- Komoran
    - 코모란. Shineware에서 개발
    - https://github.com/shin285/KOMORAN
- Okt
    - Open Korean Text, 오픈소스 한국어 분석기, 과거 트위치 형태소 분석기
    - https://github.com/open-korean-text/open-korean-text

### 2) 불용어 추가
- 데이터를 출력해 보고, 불용어를 총 50개가 될 때까지 추가해본다.
- 추가 전, 후의 모델 성능을 비교해본다.

### 3) 다른 날짜 데이터 추가
- 5월6일 데이터와 추가로 수집된 데이터를 적절히 합치려면?
- 어떻게 해야지 적절히 섞을지?

---
- 제공 받은 데이터셋 : 2020년 4월 15일자 뉴스
- 프로젝트 내의 ```code_list = [101, 102, 103, 105]```로 50개씩 다운로드

#### 그렇다면 5월 6일의 기사와 4월 15일의 기사를 합쳐서 학습을 하면 될 것이다.
---

## 1. 라이브러리 import

In [155]:
import os
from glob import glob
import pandas as pd

from konlpy.tag import Mecab, Hannanum, Kkma, Komoran, Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

import re
from tqdm.notebook import tqdm

## 2. 데이터 불러오기
- 읽어서 데이터 프레임에 저장
- 5월 6일 데이터 + 4월 15일 데이터를 합치도록 한다.

In [171]:
csv_paths = glob(os.getenv("HOME") + "/aiffel/news_crawler/*.csv")
df = pd.DataFrame()

for csv_path in csv_paths :
    df = df.append(pd.read_table(csv_path, sep=','), ignore_index=True)
    
df.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


#### 한글을 제외한 문자 제거
- 기존의 코드
```python
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']
```
- 정규식을 활용해서 태그, 한글을 제외한 문자들을 변경
    - 태그 : 공백
    - 한글+공백 제외한 문자 : 제거

In [172]:
pattern1 = re.compile("[\\n]")
pattern2 = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣' ' ]")

df['news'] = df['news'].map(lambda x : pattern1.sub(' ', x))
df['news'] = df['news'].map(lambda x : pattern2.sub('', x))
df.head()

,news,code
0,파주시청 사진제공파주시 파주시청 사진제공파주시 파주파이낸셜뉴스 강근주 기자 파주시...,사회
1,동영상 뉴스 이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...,사회
4,서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 '사회적 거리두기...,사회


#### null값 확인

In [173]:
print(df.isnull().sum())

news    0
code    0
dtype: int64


#### 중복확인
- 기존에 8827개의 기사가 6127개로 줄었다. 중복된 기사들을 제거했다.

In [174]:
# 중복된 샘플들을 제거합니다.
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  6127


In [175]:
# df['code'].value_counts().plot(kind = 'bar')
print(df.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학   1138
1     경제    901
2     사회   2222
3  생활/문화   1866


#### 불용어 파일 불러오기
- https://www.ranks.nl/stopwords/korean
    - 위에서 제공하는 불용어 활용

In [176]:
pattern = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣' ' ]")
stopwords = []

# txt파일 읽기
f = open("/home/aiffel0042/aiffel/news_crawler/stopwords/ranksnl-korean.txt")
while True:
    line = f.readline()
    if not line :
        break
    stopwords.append(line)
f.close()

print("불용어 총 갯수 : {}".format(len(stopwords)))
print(stopwords[:10])
print("\\n 태그 없애기 수행")

for i in tqdm(range(len(stopwords))):
    stopwords[i] = pattern.sub('', stopwords[i])

print(stopwords[:10])

불용어 총 갯수 : 677
['아\n', '휴\n', '아이구\n', '아이쿠\n', '아이고\n', '어\n', '나\n', '우리\n', '저희\n', '따라\n']
\n 태그 없애기 수행



['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']


## 3. 전처리 ~ 학습
- ```process``` 함수 생성
    - ```news_df``` : 뉴스 내용, 카테고리가 담긴 데이터 프레임
    - ```token``` : tokenizer선택 (Mecab, Hannanum, Kkma, Komoran, Okt / default = Mecab)
- 다음과 같이 진행
    1. input data(news_df)로 부터 입력받은 tokenizer로 기사내용을 토큰화
    2. 불용어 제거
    3. 데이터셋 나누기 (train:test = 8:2, 랜덤O)
    4. 훈련 기사 벡터화, tfidf구하기

In [177]:
def process(news_df, token='Mecab'):
    print('-'*20)
    
    # 토크나이저 설정
    if token == 'Mecab':
        tokenizer = Mecab()
        pass
    elif token == 'Hannanum':
        tokenizer = Hannanum()
        pass
    elif token == 'Kkma':
        tokenizer = Kkma()
        pass
    elif token == 'Komoran':
        tokenizer = Komoran()
        pass
    elif token == 'Okt':
        tokenizer = Okt()
        pass
    else :
        print("Wrong Tokenizer Input")
        return False
    
    print("Tokenizer : {}".format(token))
    print('-'*20)
    
    # 불용어 제거
    # 기사를 토큰으로 먼저 나누고 각 토큰과 불용어 사전을 비교
    news_article = news_df['news']
    text_data = []
    print("토큰화, 불용어 제거 진행중..")
    for sentence in tqdm(news_article):
        tmp_data = []
        # 토큰화
        tmp_data = tokenizer.morphs(sentence)
        # 불용어 제거
        tmp_data = [word for word in tmp_data if not word in stopwords]
        text_data.append(tmp_data)
    # 불용어를 제거한 기사의 토큰들이 text_data에 담기게 된다
    text_data = list(map(' '.join, text_data))
    print('불용어 제거 완료')
    
    
    # 학습, 평가 데이터 나누기
    # random_state인자를 줌으로써 랜덤하게 데이터를 나눔
    X_train, X_test, y_train, y_test = train_test_split(text_data, news_df['code'], random_state = 77)
    print('-'*20)
    
    # 개수는 어떤 토크나이저이든 같은 비율로 나누어 질 것이다
    # 그렇기 때문에 밑에서 학습을 시킬 때 Mecab을 시작으로 다른 모델을 참고하기 때문에
    # 다음과 같은 조건을 줬음
    if token == 'Mecab' :
        print('훈련용 뉴스 기사의 개수 :', len(X_train))
        print('테스트용 뉴스 기사의 개수 : ', len(X_test))
        print('훈련용 레이블의 개수 : ', len(y_train))
        print('테스트용 레이블의 개수 : ', len(y_test))

    print('-'*20)
    print('학습중')
    print('-'*20)
    
    # 카운트벡터라이저
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    
    # 카운터 벡터라이저의 결과로 tf-idf
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
    # 나이브 베이즈 분류기 수행
    # X_Train => tf-idf벡터, y_train => 레이블
#     clf = MultinomialNB().fit(X_train_tfidf, y_train)

    # 기존의 나이브 베이즈 분류기가 낮은 점수를 보여서 SGDClassifier로 학습
    clf = SGDClassifier().fit(X_train_tfidf, y_train)
    
    def tfidf_vectorizer(data):
        data_counts = count_vect.transform(data)
        data_tfidf = tfidf_transformer.transform(data_counts)
        return data_tfidf
    
    print("테스트 데이터 예측")
    
    y_pred = clf.predict(tfidf_vectorizer(X_test))
    result = metrics.classification_report(y_test, y_pred)
    print("완료\n")
    print(result)
    print('-'*30)
    print('\n\n')
    
    return result

### 학습 시작

In [181]:
tokenizers = ['Mecab', 'Hannanum', 'Kkma', 'Komoran', 'Okt'] 

reports = []

for token in tokenizers:
    reports.append(process(df, token))


--------------------
Tokenizer : Mecab
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
훈련용 뉴스 기사의 개수 : 4595
테스트용 뉴스 기사의 개수 :  1532
훈련용 레이블의 개수 :  4595
테스트용 레이블의 개수 :  1532
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.83      0.83      0.83       284
          경제       0.84      0.79      0.82       225
          사회       0.88      0.88      0.88       557
       생활/문화       0.81      0.84      0.83       466

    accuracy                           0.85      1532
   macro avg       0.84      0.84      0.84      1532
weighted avg       0.85      0.85      0.85      1532

------------------------------



--------------------
Tokenizer : Hannanum
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.82      0.87      0.84       284
          경제       0.86      0.80      0.83       225
          사회       0.88      0.87      0.88       557
       생활/문화       0.82      0.83      0.82       466

    accuracy                           0.85      1532
   macro avg       0.84      0.84      0.84      1532
weighted avg       0.85      0.85      0.85      1532

------------------------------



--------------------
Tokenizer : Kkma
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.83      0.83      0.83       284
          경제       0.84      0.80      0.82       225
          사회       0.88      0.87      0.88       557
       생활/문화       0.81      0.85      0.83       466

    accuracy                           0.85      1532
   macro avg       0.84      0.84      0.84      1532
weighted avg       0.85      0.85      0.85      1532

------------------------------



--------------------
Tokenizer : Komoran
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.83      0.85      0.84       284
          경제       0.86      0.80      0.83       225
          사회       0.88      0.88      0.88       557
       생활/문화       0.82      0.83      0.82       466

    accuracy                           0.85      1532
   macro avg       0.85      0.84      0.84      1532
weighted avg       0.85      0.85      0.85      1532

------------------------------



--------------------
Tokenizer : Okt
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.84      0.84      0.84       284
          경제       0.83      0.78      0.81       225
          사회       0.88      0.87      0.88       557
       생활/문화       0.80      0.84      0.82       466

    accuracy                           0.84      1532
   macro avg       0.84      0.83      0.84      1532
weighted avg       0.84      0.84      0.84      1532

------------------------------





In [180]:
for token, report in list(zip(tokenizers, reports)):
    print(token)
    print('')
    print(report)
    print('-'*20+'\n\n')

Mecab

              precision    recall  f1-score   support

       IT/과학       0.83      0.83      0.83       284
          경제       0.84      0.80      0.82       225
          사회       0.88      0.88      0.88       557
       생활/문화       0.81      0.84      0.83       466

    accuracy                           0.85      1532
   macro avg       0.84      0.84      0.84      1532
weighted avg       0.85      0.85      0.85      1532

--------------------


Hannanum

              precision    recall  f1-score   support

       IT/과학       0.82      0.87      0.84       284
          경제       0.85      0.80      0.83       225
          사회       0.89      0.87      0.88       557
       생활/문화       0.82      0.84      0.83       466

    accuracy                           0.85      1532
   macro avg       0.85      0.84      0.84      1532
weighted avg       0.85      0.85      0.85      1532

--------------------


Kkma

              precision    recall  f1-score   support

      

### +. 점수를 더 높일 수 있을까?
#### 데이터의 비율을 일정하게 맞추고 해보았다.
```
    code  count
0  IT/과학   1138
1     경제    901
2     사회   2222
3  생활/문화   1866
```
- 경제에 대한 데이터는 적기 때문에 다른 분야의 데이터수를 1000개로 맞추어 주었다.

In [148]:
df.head()

,news,code
0,파주시청 사진제공파주시 파주시청 사진제공파주시 파주파이낸셜뉴스 강근주 기자 파주시...,사회
1,동영상 뉴스 이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...,사회
4,서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 '사회적 거리두기...,사회


In [149]:
so_df = df[df['code'] == '사회']
so_df = so_df.iloc[:1000].reset_index(drop=True)
print(len(so_df))
so_df.head()

1000


,news,code
0,파주시청 사진제공파주시 파주시청 사진제공파주시 파주파이낸셜뉴스 강근주 기자 파주시...,사회
1,동영상 뉴스 이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...,사회
4,서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 '사회적 거리두기...,사회


In [150]:
ec_df = df[df['code'] == '경제']
ec_df = ec_df.iloc[:1000].reset_index(drop=True)
print(len(ec_df))
ec_df.head()

901


,news,code
0,제대 총선 실시간 개표 현황 및 결과 보기 총선에서 여당이 다시 한 번 승리를 거...,경제
1,뉴욕뉴시스지난 일 뉴욕 증권거래소 건물에 미국 국기가 게양되어 있다 신종 코로나바이...,경제
2,부산지역 주유소에서 판매하는 기름값이 휘발유는 평균 원대 경유는 원대에 돌입했다일 ...,경제
3,담배업계가 소비자의 취향을 저격한 다양한 담배 신제품들을 잇달아 선보이고 있다이전까...,경제
4,이데일리 박태진 기자 다음은 일자 이데일리신문 주요 기사다면민주 연승 자만 말고 협...,경제


In [151]:
cl_df = df[df['code'] == '생활/문화']
cl_df = cl_df.iloc[:1000].reset_index(drop=True)
print(len(cl_df))
cl_df.head()

1000


,news,code
0,헤럴드경제고승희 기자 문화체육관광부는 한국문화예술위원회 기 비상임위원으로 다문화극단...,생활/문화
1,동영상 뉴스 오늘 서쪽 지방과 동쪽 지방의 온도 차가 제법 컸습니다서쪽 지방은 강...,생활/문화
2,나는 지난 여년간 협력이라는 말을 매일 마주하며 살고 있다 남북관계 현장에서 젊음을...,생활/문화
3,아가타 파리 아가타 파리 아가타 파리 아가타 파리 아가타 파리 가 시그니...,생활/문화
4,'까칠남녀'에 출연한 정영진 '까칠남녀'에 출연한 정영진 '까칠남녀'에 출연...,생활/문화


In [152]:
it_df = df[df['code'] == 'IT/과학']
it_df = it_df.iloc[:1000].reset_index(drop=True)
print(len(it_df))
it_df.head()

1000


,news,code
0,서울뉴시스 오동현 기자 모바일 게임 기업 컴투스는 모바일 야구 게임 컴투스프로야...,IT/과학
1,연합뉴스 연합뉴스 요즘은 잔인한 날리프트도 앞서 명 일시해고뉴욕연합뉴스 이귀원 특...,IT/과학
2,지난 년 포항공대에 구축한 세대 선형 방사광가속기 연합뉴스 지난 년 포항공대에 구축...,IT/과학
3,나주 방사광 가속기 조감도 전남도 제공 나주 방사광 가속기 조감도 전남도 제공 치...,IT/과학
4,서울뉴시스 오동현 기자 펍지주식회사가 올해 첫 '플레이어언노운스 배틀그라운드' 글...,IT/과학


In [153]:
final_df = pd.DataFrame()
# df = df.append(pd.read_table(csv_path, sep=','), ignore_index=True)
final_df = final_df.append(so_df, ignore_index=True)
final_df = final_df.append(ec_df, ignore_index=True)
final_df = final_df.append(cl_df, ignore_index=True)
final_df = final_df.append(it_df, ignore_index=True)

final_df.reset_index(drop=True)

,news,code
0,파주시청 사진제공파주시 파주시청 사진제공파주시 파주파이낸셜뉴스 강근주 기자 파주시...,사회
1,동영상 뉴스 이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...,사회
4,서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 '사회적 거리두기...,사회
...,...,...
3896,전 세계가 신종 코로나바이러스 감염증코로나 확산 대응의 모범국가로 한국을 꼽고 있는...,IT/과학
3897,애플 모빌리티 트렌드 리포트에서 나타난 국내 이동량 추이 애플 제공 애플 모빌리티 ...,IT/과학
3898,일 에서 차 온라인 개학대비 학습관리 시스템 합동 점검 회의를 진행했다 지디넷코리...,IT/과학
3899,축구톡톡 편 공개전북 현대 제공 축구톡톡 편 공개전북 현대 제공 서울연합뉴스 안홍...,IT/과학


In [159]:
reports = []

for token in tokenizers:
    reports.append(process(token, final_df))

--------------------
Tokenizer : Mecab
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
훈련용 뉴스 기사의 개수 : 2925
테스트용 뉴스 기사의 개수 :  976
훈련용 레이블의 개수 :  2925
테스트용 레이블의 개수 :  976
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.83      0.86      0.84       225
          경제       0.85      0.84      0.84       244
          사회       0.82      0.79      0.81       268
       생활/문화       0.77      0.79      0.78       239

    accuracy                           0.82       976
   macro avg       0.82      0.82      0.82       976
weighted avg       0.82      0.82      0.82       976

------------------------------



--------------------
Tokenizer : Hannanum
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.82      0.86      0.84       225
          경제       0.87      0.85      0.86       244
          사회       0.82      0.78      0.80       268
       생활/문화       0.77      0.78      0.78       239

    accuracy                           0.82       976
   macro avg       0.82      0.82      0.82       976
weighted avg       0.82      0.82      0.82       976

------------------------------



--------------------
Tokenizer : Kkma
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.81      0.86      0.84       225
          경제       0.88      0.84      0.86       244
          사회       0.81      0.79      0.80       268
       생활/문화       0.76      0.77      0.77       239

    accuracy                           0.82       976
   macro avg       0.82      0.82      0.82       976
weighted avg       0.82      0.82      0.82       976

------------------------------



--------------------
Tokenizer : Komoran
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.82      0.87      0.85       225
          경제       0.86      0.85      0.86       244
          사회       0.81      0.79      0.80       268
       생활/문화       0.77      0.76      0.76       239

    accuracy                           0.82       976
   macro avg       0.82      0.82      0.82       976
weighted avg       0.82      0.82      0.82       976

------------------------------



--------------------
Tokenizer : Okt
--------------------
토큰화, 불용어 제거 진행중..



불용어 제거 완료
--------------------
--------------------
학습중
--------------------
테스트 데이터 예측
완료

              precision    recall  f1-score   support

       IT/과학       0.84      0.89      0.86       225
          경제       0.88      0.85      0.86       244
          사회       0.84      0.79      0.82       268
       생활/문화       0.77      0.80      0.79       239

    accuracy                           0.83       976
   macro avg       0.83      0.83      0.83       976
weighted avg       0.83      0.83      0.83       976

------------------------------





In [165]:
print("-------------------사용한 Tokenizer : {}-------------------".format(tokenizers[4]))
print(reports[4])

-------------------사용한 Tokenizer : Okt-------------------
              precision    recall  f1-score   support

       IT/과학       0.84      0.89      0.86       225
          경제       0.88      0.85      0.86       244
          사회       0.84      0.79      0.82       268
       생활/문화       0.77      0.80      0.79       239

    accuracy                           0.83       976
   macro avg       0.83      0.83      0.83       976
weighted avg       0.83      0.83      0.83       976



# 정리
1. Tokenizer 5가지 비교, 불용어 제거를 위해서 677개의 불용어 파일을 읽어들임
2. 5월6일의 데이터, 4월 15일의 데이터를 합쳐서 사용.
    - 기존에 8827개의 기사에서 중복된 기사를 제거 후 6127개로 줄었다.
3. 데이터 6127개로 학습 및 분류
    - 토크나이저 변경, f1score : okt를 제외한 토크나이저는 모두 0.85의 점수가 나왔고 okt는 0.84가 나왔다.
    - 나이브 베이즈 분류기 -> SGD 분류기 : 기존의 나이브 베이즈 분류기의 점수는 0.7의 후반대에 머물렀지만 분류기를 다른 것으로 바꿔보았고 높은 점수가 나오게 되었다.
4. 데이터의 비율을 맞춰서 한번더 확인
    - 일부 카테고리의 경우는 점수가 오르는 것을 확인했지만 반면에 ```생활/문화```의 경우 점수가 떨어지는 것을 확인
    - 일반적으로 데이터가 더 많을 수록 분류를 잘한것이라고 예측할 수 있을 거 같다.

#### 주말에 복습할 내용
- 데이터 수를 늘려본다.
- 나이브 베이즈로도 높은 점수를 받도록 해보고
    - 나이브 베이즈 분류기의 경우 데이터가 상대적으로 적은 ```경제``` 분야의 기사 정확도가 많이 떨어졌다.
    - 반면에 SGD 분류기로 확인할 경우 대부분 비슷하게 높은 점수가 나오는 것을 확인하였다
    - 랜덤포레스트, SVM등 이전에 배웠던 분류기를 바꿔가면서 다른 결과도 확인하고 분류기마다 특성을 좀더 자세하게 정리를 해보도록 한다.
- 데이터의 균형 및 불용어에 따라서 점수가 어떻게 바뀌는지 확인 해보자